gsheet_api_check() function. It looks for an existing token.pickle file (which stores our user access and refresh tokens). If no token.pickle file is found, the function will prompt you to log into your Google Gmail account. The credentials.json must be present in your working directory to initiate token.pickle creation/refresh. The function will specify and return the credentials we’ll use to make API calls #google

Next we’ll define a function that makes the API call and pulls the data we want from Google Sheets. The pull_sheet_data() function establishes the API call and pulls the data we want. If no data is found the function will print “No data found.”, otherwise it will confirm the data has been retrieved by printing “COMPLETE: Data Copied” and return our data #googled it too

In [33]:
import pandas as pd
import pickle
import matplotlib as plt
import os.path
import numpy as np
import datetime
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data



Next we need to find and copy the ID of the spreadsheet. 
Second, we’ll need the name of the spreadsheet tab from which we’ll pull the data from.

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']    
SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'
RANGE_NAME = 'transactions'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
transactions = pd.DataFrame(data[1:], columns=data[0])

RANGE_NAME = 'clients'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
clients = pd.DataFrame(data[1:], columns=data[0])

RANGE_NAME = 'managers'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
managers = pd.DataFrame(data[1:], columns=data[0])

RANGE_NAME = 'leads'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
leads = pd.DataFrame(data[1:], columns=data[0])


COMPLETE: Data copied
COMPLETE: Data copied
COMPLETE: Data copied
COMPLETE: Data copied


In [3]:
#getting info about our 4 datasets
transactions.head(1)

,transaction_id,created_at,m_real_amount,l_client_id
0,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,31,8a805e60-6fd6-11e7-80fc-c412f533dba1


In [4]:
clients.head(1)

,client_id,created_at,l_manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1


In [5]:
managers.head(1)

,manager_id,d_manager,d_club
0,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2


In [6]:
leads.head(1)

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1


In [7]:
#merge this four datasets and rename some of them. also drop duplicate columns

In [8]:
df = leads.merge(
    managers, left_on='l_manager_id', right_on='manager_id', how='outer').merge(
    clients, left_on='l_client_id', right_on='client_id', how='outer').merge(
    transactions, on='l_client_id', how='outer'
)
df = df.rename(columns={"l_manager_id_x": "l_manager_id", "created_at_y": "date_of_registration",
                        'created_at_x':'created_at','created_at':'date_of_transaction' })
df = df.drop(['manager_id', 'l_manager_id_y', 'client_id'],axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 105201 entries, 0 to 105200
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   lead_id               4295 non-null   object
 1   created_at            4295 non-null   object
 2   d_utm_medium          4295 non-null   object
 3   d_utm_source          4295 non-null   object
 4   l_manager_id          4295 non-null   object
 5   l_client_id           31616 non-null  object
 6   d_manager             3670 non-null   object
 7   d_club                3670 non-null   object
 8   date_of_registration  77013 non-null  object
 9   transaction_id        29289 non-null  object
 10  date_of_transaction   29289 non-null  object
 11  m_real_amount         29289 non-null  object
dtypes: object(12)
memory usage: 10.4+ MB


In [9]:
#get info about merge table. Need to provide necessary types for digits and datetime

In [10]:
df['m_real_amount'] = pd.to_numeric(df['m_real_amount'])
df['created_at'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['date_of_registration'] = pd.to_datetime(df['date_of_registration'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['date_of_transaction'] = pd.to_datetime(df['date_of_transaction'], format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [11]:
#looking for duplicates and wrong names

In [12]:
df['d_utm_source'].unique()

array(['viber', 'insta', '', 'instagram', 'utm_source', 'vkontakte',
       'google', 'yandex', 'sms', 'vk', 'ycard',
       'ycard#!/tproduct/225696739-1498486363994', 'ig', nan],
      dtype=object)

In [13]:
#rename duplicates to correctly estimate data
df['d_utm_source'] = df['d_utm_source'].replace('vkontakte', 'vk')
df['d_utm_source'] = df['d_utm_source'].replace('', np.nan)
df['d_utm_source'] = df['d_utm_source'].replace('insta', 'instagram')
df['d_utm_source'] = df['d_utm_source'].replace('ig', 'instagram')

In [14]:
df['d_utm_source'].unique()

array(['viber', 'instagram', nan, 'utm_source', 'vk', 'google', 'yandex',
       'sms', 'ycard', 'ycard#!/tproduct/225696739-1498486363994'],
      dtype=object)

In [15]:
df['d_utm_source'] = df['d_utm_source'].replace('ycard#!/tproduct/225696739-1498486363994', 'ycard')
df.replace('00000000-0000-0000-0000-000000000000', np.nan, inplace=True)

In [16]:
df['d_utm_source'].unique()

array(['viber', 'instagram', nan, 'utm_source', 'vk', 'google', 'yandex',
       'sms', 'ycard'], dtype=object)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105201 entries, 0 to 105200
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   lead_id               4295 non-null   object        
 1   created_at            4295 non-null   datetime64[ns]
 2   d_utm_medium          4295 non-null   object        
 3   d_utm_source          2763 non-null   object        
 4   l_manager_id          3761 non-null   object        
 5   l_client_id           30792 non-null  object        
 6   d_manager             3670 non-null   object        
 7   d_club                3670 non-null   object        
 8   date_of_registration  47612 non-null  datetime64[ns]
 9   transaction_id        29289 non-null  object        
 10  date_of_transaction   29289 non-null  datetime64[ns]
 11  m_real_amount         29289 non-null  float64       
dtypes: datetime64[ns](3), float64(1), object(8)
memory usage: 10.4+ MB


In [18]:
#looking for clients who bought within 1 week after lead
df['within_week'] = (df['date_of_transaction'] - datetime.timedelta(days=7)) <= (df['created_at'])

In [19]:
#looking for trash leads
df['no_clients_of_lead'] = (df['lead_id'].notna()) & (df['l_client_id'].isna())

In [20]:
#looking for new leads (no transactions and no leads before)
df['new_leads'] = (df['lead_id'].isna() & df['transaction_id'].isna())

In [21]:
#looking for leads with return of new clients (no lead before but with transaction exists)
df['leads_return'] = ((df['lead_id'].notnull()) & (df['transaction_id'].isna()))

In [22]:
#looking for leads with return of old clients (transaction dates are older than 3 month)
df['three_month'] = ((df['created_at'] - datetime.timedelta(days=90)) >= (df['date_of_transaction'])) & (
    df['lead_id'].notnull())

In [23]:
#looking for new clients who bougth within 1 week after lead but didnt buy earlier
df['new_clients'] = (df['within_week'] & (~(df['transaction_id'].notnull())))


In [24]:
#making profit column equal to 'm_real_amount' if transactions were within 1 week
df.loc[df['within_week'] == True, 'profit'] = df['m_real_amount']

In [29]:
#create pivot table from all columns i've made and 3 old columns as a dimensions
df_pivot = df.pivot_table(index = ['d_utm_source', 'd_club', 'd_manager'], 
                          values = ['lead_id',
                                    'within_week',
                                    'no_clients_of_lead',
                                    'new_leads',
                                    'leads_return',
                                    'three_month',
                                    'new_clients',
                                    'profit'
                                   ], 
                          aggfunc =({'lead_id':'count',
                                    'within_week':['sum'],
                                    'no_clients_of_lead':'sum',
                                     'new_leads':'sum',
                                     'leads_return':'sum',
                                     'three_month':'sum',
                                     'new_clients':'sum',
                                     'profit':'sum'
                                     
                                    ,}))

In [30]:
names = ['lead_id_count',
         'leads_return',
         'new_clients',
         'new_leads',
         'no_clients_of_lead', 
         'profit', 
         'three_month', 
         'within_week']
df_pivot.set_axis(names,axis = 'columns',inplace = True)
df_pivot

lead_id_count  leads_return  new_clients  \
d_utm_source d_club  d_manager                                               
google       club #1 manager #1              21            17            0   
                     manager #7               7             6            0   
             club #2 manager #10             43             8            0   
                     manager #4              14             5            0   
                     manager #8              17            11            0   
...                                         ...           ...          ...   
ycard        club #1 manager #7               3             2            0   
             club #2 manager #4               2             1            0   
                     manager #8               4             3            0   
             club #3 manager #13              1             1            0   
             club #4 manager #2               4             4            0   

                                  new_leads  no_clients_of_lead    profit  \
d_utm_source d_club  d_manager                                              
google       club #1 manager #1           0                   9   64811.0   
                     manager #7           0                   1    9212.0   
             club #2 manager #10          0                   1  341597.0   
                     manager #4           0                   0   69703.0   
                     manager #8           0                   1  217806.0   
...                                     ...                 ...       ...   
ycard        club #1 manager #7           0                   0   22628.0   
             club #2 manager #4           0                   0    3670.0   
                     manager #8           0                   0   79395.0   
             club #3 manager #13          0                   0       0.0   
             club #4 manager #2           0                   1       0.0   

                                  three_month  within_week  
d_utm_source d_club  d_manager                              
google       club #1 manager #1             0            3  
                     manager #7             0            1  
             club #2 manager #10           21           31  
                     manager #4             2            7  
                     manager #8             0            6  
...                                       ...          ...  
ycard        club #1 manager #7             0            1  
             club #2 manager #4             0            1  
                     manager #8             0            1  
             club #3 manager #13            0            0  
             club #4 manager #2             0            0  

[84 rows x 8 columns]